In [52]:
import numpy as np
import pandas as pd
import random

In [53]:
MUTATION_TYPES = {"begin_end": 0, "middle" : 1, "cikcak" : 2}

In [54]:
def get_reference(file_name):
    f = open(file_name, "r")
    print(f.readline().strip())
    transcript = f.readline().strip()
    print(transcript)
    print('Duljina transkripta: ' + str(len(transcript)))
    return transcript

In [55]:
letters = ['A', 'C', 'G', 'T']
def get_mutation(position):
    mutation = random.sample(letters, 1)[0]
    while(mutation == transcript[position]):
        mutation = random.sample(letters, 1)[0]
    return mutation

In [56]:
def create_mutations(positions):
    mutations = []
    for position in positions:
        mutations.append((position, get_mutation_tetra(position)))
    return mutations

In [57]:
def get_mutation_positions(transcript, mutation_type, min_num_mutations, max_num_mutations):
    mutation_positions = []
    l = int(random.uniform(min_num_mutations, max_num_mutations))
    if mutation_type == 0: #mutations begin_end
        b = int(random.uniform(10, l - 10))
        e = l - b
        mutation_positions.extend(random.sample(range(30, 30 + l), b))
        mutation_positions.extend(random.sample(range(len(transcript) - 30 - l, len(transcript) - 30), e))
        print('Mutacije na početku i na kraju')
        print(mutation_positions)
    elif mutation_type == 1: #mutations middle
        l = int(random.uniform(min_num_mutations, max_num_mutations))
        mutation_positions = random.sample(range(int(len(transcript)/3), int(2*len(transcript)/3)), l)
        print('Mutacije na sredini ' + str(int(len(transcript)/3)) + ' - ' + str(int(2*len(transcript)/3)))
        print(mutation_positions)
    elif mutation_type == 2:
        pos = 30
        l = 0
        while pos < len(transcript) - 30:
            num_mutations = int(random.uniform(3, 7))
            l += num_mutations
            mutation_positions.extend(random.sample(range(pos - num_mutations, pos + num_mutations), num_mutations))
            pos += 20
        print('Svakih 20ak baza nekoliko mutacija')
        print(mutation_positions)
    
    return mutation_positions

In [58]:
def generate_reads(reads, number_of_reads, p, cluster_mutations, transcript):
    for i in range(number_of_reads):
        cluster = random.sample(range(K), 1)[0]
        must_have_mutations = cluster_mutations[cluster]
        read_start = 0
        read_end = len(transcript)
        read_length = read_end - read_start
        read = transcript[read_start : read_end + 1]
        read_mmap = [0] * len(read)
        for (position, mutation) in must_have_mutations:
            if position >= read_start and position <= read_end:
                index_on_read = position - read_start
                read= read[:index_on_read] + mutation + read[index_on_read + 1:]
                read_mmap[index_on_read] = 1
        counter = 0
        for j in range(read_length):   
            mutation_percentage = random.random()
            if mutation_percentage < p:
                counter = counter + 1
                read = read[:j] + getMutation(j + read_start) + read[j + 1:]
                read_mmap[j] = 1
        reads[(read, read_start, read_end)] = (cluster, read_mmap)

In [59]:
def check_clusters_distribution(K, reads):
    num_reads_per_cluster = []
    for k in range(K):
        num_reads_per_cluster.append(0)
    for r in reads:
        num_reads_per_cluster[reads[r][0]] = num_reads_per_cluster[reads[r][0]] + 1

    print("Clusters distribution: " + str(num_reads_per_cluster))

In [60]:
def reads_to_file(reads, file_name):
    f = open(file_name, "w")
    for i, r in enumerate(reads):
        name = "read_" + str(i)
        mmap = ""
        for modif in reads[r][1]:
            mmap += str(modif) + " "
        mmap = mmap[:-1]
        f.write(name + "\t" + mmap + "\t" + str(reads[r][0]) + "\n")

# simulated reads for tetrahymena

In [61]:
transcript = get_reference("tetra.fasta")
cluster_mutations = []
min_num_mutations = 80
max_num_mutations = 110

mutation_positions_cluster_1 = get_mutation_positions(transcript, MUTATION_TYPES["begin_end"], min_num_mutations, max_num_mutations)
mutations_cluster1 = create_mutations(mutation_positions_cluster_1)
cluster_mutations.append(mutations_cluster1)

mutation_positions_cluster_2 = get_mutation_positions(transcript, MUTATION_TYPES["middle"], min_num_mutations, max_num_mutations)
mutations_cluster2 = create_mutations(mutation_positions_cluster_2)
cluster_mutations.append(mutations_cluster2)

mutation_positions_cluster_3 = get_mutation_positions(transcript, MUTATION_TYPES["cikcak"], min_num_mutations, max_num_mutations)
mutations_cluster3 = create_mutations(mutation_positions_cluster_3)
cluster_mutations.append(mutations_cluster3)

K = len(cluster_mutations)

number_of_reads = 7000  #at the end it will be smaller number
p = 0.01   
reads = {}

generate_reads(reads, number_of_reads, p, cluster_mutations, transcript)

check_clusters_distribution(K, reads)

reads_to_file(reads, "tetra_simulated_data.txt")

>tetra
GGCTCTCTAAATAGCAATATTTACCTTTGGAGGGAAAAGTTATCAGGCATGCACCTGGTAGCTAGTCTTTAAACCAATAGATTGCATCGGTTTAAAAGGCAAGACCGTCAAATTGCGGGAAAGGGGTCAACAGCCGTTCAGTACCAAGTCTCAGGGGAAACTTTGAGATGGCCTTGCAAAGGGTATGGTAATAAGCTGACGGACATGGTCCTAACCACGCAGCCAAGTCCTAAGTCAACAGATCTTCTGTTGATATGGATGCAGTTCACAGACTAAATGTCGGTCGGGGAAGATGTATTCTTCTCATAAGATATAGTCGGACCTCTCCTTAATGGGAGCTAGCGGATGAAGTGATGCAACACTGGAGCCGCTGGGAACTAATTTGTATGCGAAAGTATATTGATTAGTTTTGGAGTACTCG
Duljina transkripta: 421
Mutacije na početku i na kraju
[49, 100, 102, 48, 52, 47, 114, 74, 75, 109, 78, 43, 53, 57, 94, 36, 92, 95, 86, 90, 101, 66, 82, 77, 113, 59, 46, 97, 61, 51, 33, 87, 45, 99, 60, 91, 44, 65, 58, 63, 108, 40, 112, 93, 310, 384, 386, 357, 371, 312, 339, 328, 347, 353, 365, 327, 305, 343, 338, 332, 319, 376, 307, 369, 363, 324, 331, 389, 314, 388, 345, 336, 322, 352, 323, 333, 320, 329, 317, 377, 350, 325, 359, 308, 304, 337, 346]
Mutacije na sredini 140 - 280
[206, 164, 210, 228, 201, 172, 244, 251, 197, 221, 209, 227, 152, 153, 246, 263, 187,

# simulated data for riboswitches

In [62]:
transcript = get_reference("tpp_reference.fasta")
cluster_mutations = []
min_num_mutations = 80
max_num_mutations = 110

mutation_positions_cluster_1 = get_mutation_positions(transcript, MUTATION_TYPES["begin_end"], min_num_mutations, max_num_mutations)
mutations_cluster1 = create_mutations(mutation_positions_cluster_1)
cluster_mutations.append(mutations_cluster1)

mutation_positions_cluster_2 = get_mutation_positions(transcript, MUTATION_TYPES["middle"], min_num_mutations, max_num_mutations)
mutations_cluster2 = create_mutations(mutation_positions_cluster_2)
cluster_mutations.append(mutations_cluster2)

# mutation_positions_cluster_3 = get_mutation_positions(transcript, MUTATION_TYPES["cikcak"], min_num_mutations, max_num_mutations)
# mutations_cluster3 = create_mutations(mutation_positions_cluster_3)
# cluster_mutations.append(mutations_cluster3)

K = len(cluster_mutations)

number_of_reads = 7000  #at the end it will be smaller number
p = 0.01   
reads = {}

generate_reads(reads, number_of_reads, p, cluster_mutations, transcript)

check_clusters_distribution(K, reads)

reads_to_file(reads, "ribo_simulated_data.txt")

>TPP_1_tenA
TAACCACTAGGGGTGTCCTTCATAAGGGCTGAGATAAAAGTGTGACTTTTAGACCCTCATAACTTGAACAGGTTCAGACCTGCGTAGGGAAGTGGAGCGGTATTTGTGTTATTTTACTATGCCAATTCCAAACCACTTTTCCTTGCGGGAAAGTGGTTTTTTTATTTTCAGAGGGGGAATGATTTGTGAAGTTTTCAGAAGAATGCCGCAGTGCAGCCGCAGAATGGTGGGAGGGGAGCTTTGTCCATCCGTTCGTTCAAGGAATCGGTGACGGAACGCTTCCGATTGACCGTTTTAAATACTACGTACTTCAGGATTCCTATTATTTAACGCATTTTGCAAAGGTGCAGTCATTTGGTGCCGCTTATGCGAAGGATCTTTATACAACGGGGCGGATGGCAAGCCATGCCCAAGGTACATATGAGGCGGAAATGGCGCTTCATCGCGAGTTTGCCGAGCTGTTGGAAATCAGCGAGGAAGAGCGTAAGGCGTTTAAGCCGTCTCCTACGGCGTACTCTTATACATCCCATATGTACCGTTCGGTTTTGAGCGGGAATTTCGCAGAAATCTTAGCGGCCCTGCTGCCATGCTATTGGCTCTATTACGAGGTTGGTGAGAAATTGCTGCATTGTGATCCGG
Duljina transkripta: 639
Mutacije na početku i na kraju
[73, 83, 89, 30, 86, 47, 48, 69, 101, 43, 76, 56, 87, 38, 65, 79, 45, 85, 98, 90, 77, 105, 107, 74, 75, 102, 80, 99, 55, 104, 36, 84, 59, 51, 78, 31, 54, 46, 62, 93, 40, 110, 41, 61, 576, 556, 578, 535, 577, 546, 586, 529, 595, 588, 572, 605, 551, 534, 543, 582, 579, 527, 528, 573, 574, 599,